In [ ]:
from langgraph.graph import StateGraph, START, END
from langchain.chat_models import init_chat_model
from langgraph.graph.message import MessagesState

llm = init_chat_model("openai:gpt-4o-mini")

In [62]:
class State(MessagesState):
    custom_stuff: str


graph_builder = StateGraph(State)

In [63]:
def chatbot(state: State):
    response = llm.invoke(state["messages"])
    return {"messages": [response]}

In [64]:
graph_builder.add_node("chatbot", chatbot)


graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

graph = graph_builder.compile()

graph.invoke(
    {
        "messages": [
            {"role": "user", "content": "how are you?"},
        ]
    }
)

{'messages': [HumanMessage(content='how are you?', additional_kwargs={}, response_metadata={}, id='a8e0cb29-f37d-4bbf-ad60-5773c98013d9'),
  AIMessage(content="I'm just a program, so I don't have feelings, but I'm here and ready to help you! How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 11, 'total_tokens': 38, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_8bda4d3a2c', 'id': 'chatcmpl-CCqlrxJlP3lDhVlg9CsYmrbgoe5U0', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--9094ada1-b07c-49ef-87d3-7287e2dbe860-0', usage_metadata={'input_tokens': 11, 'output_tokens': 27, 'total_tokens': 38, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'outp